In [1]:
# Dependencies
import requests
from pprint import pprint
import csv
import json
import datetime
import time
import pandas as pd

In [7]:
time_list = ['20190101+TO+20190531', '20120101+TO+20121231',
             '20180101+TO+20180531', '20180601+TO+20181231',
             '20170101+TO+20170531', '20170601+TO+20171231',
             '20160101+TO+20160531', '20160601+TO+20161231',
             '20150101+TO+20150531', '20150601+TO+20151231',
             '20140101+TO+20140531', '20140601+TO+20141231',
             '20130101+TO+20130531', '20130601+TO+20131231',
            ]
frames = []

for time_item in time_list:
    print (time_item)
    url = "https://api.fda.gov/food/enforcement.json?search=report_date:[" + time_item +"]&limit=99"
    print (url)
    respond = requests.get(url).json()
    df_temp = pd.DataFrame(respond["results"])
    df_clean = df_temp[['state', 'city', 'report_date', 'product_description', 'product_quantity', 'reason_for_recall', 'recalling_firm', 'recall_number']]
    frames.append(df_clean)
    
result = pd.concat(frames)

20190101+TO+20190531
https://api.fda.gov/food/enforcement.json?search=report_date:[20190101+TO+20190531]&limit=99
20120101+TO+20121231
https://api.fda.gov/food/enforcement.json?search=report_date:[20120101+TO+20121231]&limit=99
20180101+TO+20180531
https://api.fda.gov/food/enforcement.json?search=report_date:[20180101+TO+20180531]&limit=99
20180601+TO+20181231
https://api.fda.gov/food/enforcement.json?search=report_date:[20180601+TO+20181231]&limit=99
20170101+TO+20170531
https://api.fda.gov/food/enforcement.json?search=report_date:[20170101+TO+20170531]&limit=99
20170601+TO+20171231
https://api.fda.gov/food/enforcement.json?search=report_date:[20170601+TO+20171231]&limit=99
20160101+TO+20160531
https://api.fda.gov/food/enforcement.json?search=report_date:[20160101+TO+20160531]&limit=99
20160601+TO+20161231
https://api.fda.gov/food/enforcement.json?search=report_date:[20160601+TO+20161231]&limit=99
20150101+TO+20150531
https://api.fda.gov/food/enforcement.json?search=report_date:[20150

In [11]:
result.sort_values(by=['report_date'])
result = result.rename(columns={"recalling_firm":"brand"})

In [14]:
brands = list(result.brand.unique())
brands_df = pd.DataFrame(brands, columns=['brand'])
brands_df

,brand
0,Whole Foods Market
1,CHICAGO BAR COMPANY LLC RXBAR
2,Hy-Vee Stores Inc
3,THRIVE MARKET
4,Oskri Corp.
5,Sysco Charlotte
6,Root 23 LLC
7,"Ardent Mills, LLC"
8,Ventura Foods Llc
9,"Diamond Crystal Brands, Inc."


In [20]:
df = pd.read_csv('en.openfoodfacts.org.products.tsv', sep='\t', header=0)

In [21]:
df = df[['product_name', 'brands','ingredients_text', 'countries']]
df = df.loc[df['countries'] == 'US', :]
df = df[['product_name', 'brands','ingredients_text']]
df = df.dropna()
df = df.reset_index(drop=True)
brands = list(df.brands.unique())
brands_df_new = pd.DataFrame(brands, columns=['brand'])


In [22]:
brands_df_new

,brand
0,Torn & Glasser
1,Grizzlies
2,Bob's Red Mill
3,Unfi
4,Lundberg
5,Daddy's Muesli
6,Equal Exchange
7,Napa Valley Naturals
8,Gardentime
9,Sunridge


In [28]:
total_brand_list = pd.concat([brands_df, brands_df_new])

In [29]:
total_brand_list.nunique()
brands = list(total_brabd_list.brand.unique())

In [30]:
id_brand_table = pd.DataFrame(brands, columns=['brand'])

In [32]:
id_brand_table.head()

,brand
0,Whole Foods Market
1,CHICAGO BAR COMPANY LLC RXBAR
2,Hy-Vee Stores Inc
3,THRIVE MARKET
4,Oskri Corp.
